In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install pm4py

     |████████████████████████████████| 337kB 2.8MB/s 
     |████████████████████████████████| 48.5MB 38.9MB/s 
     |████████████████████████████████| 13.6MB 11.0MB/s 
     |████████████████████████████████| 27.4MB 30.2MB/s 
     |████████████████████████████████| 1.2MB 25.9MB/s 
  Created wheel for pulp: filename=PuLP-1.6.10-cp36-none-any.whl size=12269902 sha256=cd3fee3bda6fc4d7be036d54864e5ad0c7a4049fbe4e9c69fbfde7d3c9f01771
  Stored in directory: /root/.cache/pip/wheels/5e/76/77/e28b22219e46e3b4b033f02e8b36b2770ae545bdcf60c2b224
  Created wheel for ciso8601: filename=ciso8601-2.1.1-cp36-cp36m-linux_x86_64.whl size=26512 sha256=887ade0702e40ae8a4f128e7fc04387f2a118f0aaf53449602f027fb47ec50ea
  Stored in directory: /root/.cache/pip/wheels/72/cd/ca/9058cc007cf5b7110011a6879defcb5486e7b987e7dcf077c2
Successfully built pulp ciso8601
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1
  Found existing installation:

In [1]:
!ls '/content/drive/My Drive/Eventlog/Benchmark/seperate file/WRI'


BPIC12.xes     BPIC13_i.xes	       RTFMP.xes
BPIC13_cp.xes  BPI_Challenge_2012.xes  SEPSIS.xes


In [0]:
#Importing library to read log file
from pm4py.objects.log.importer.xes import factory as xes_import_factory
from pm4py.objects.log.exporter.csv import factory as csv_exporter
from pm4py.objects.conversion.log import factory as conversion_factory
from pm4py.objects.log.exporter.xes import factory as xes_exporter
from pm4py.algo.filtering.pandas.variants import variants_filter

from datetime import datetime
import pandas as pd
import operator
import re

In [0]:
#Reading XES fileafter 

filename = '/content/drive/My Drive/Eventlog/Benchmark/seperate file/WRI/SEPSIS.xes'
log = xes_import_factory.apply(filename)

#Converting XES to CSV file
fileName = "/content/drive/My Drive/Eventlog/Benchmark/seperate file/no_need9.csv"

csv_exporter.export(log, fileName)

# Reading the Converted CSV file
df = pd.read_csv(fileName)

# Filtering only three columns
df_clean = df.loc[:,['case:concept:name','concept:name','time:timestamp']]

In [20]:
df_clean.head()
#len(df_clean)

case:concept:name     concept:name             time:timestamp
0                 A  ER Registration  2014-10-22 11:15:41+02:00
1                 A       Leucocytes  2014-10-22 11:27:00+02:00
2                 A              CRP  2014-10-22 11:27:00+02:00
3                 A       LacticAcid  2014-10-22 11:27:00+02:00
4                 A        ER Triage  2014-10-22 11:33:37+02:00

In [80]:
c = 0
for i in df_clean.loc[:,"concept:name"].unique():
  c +=1
print(c)

16


In [0]:
act_rename = ["A01","A02","A03","A04","A05","A06","A07","A08","A09","A10","A11","A12","A13","A14","A15","A16"]                     
act_dic = {}
c = 0
for i in df_clean.loc[:,"concept:name"].unique():
  act_dic[i] = act_rename[c]
  c +=1
  
def activity_change(row):
  rename = act_dic[row]
  return rename
df_clean["concept:name"] = df_clean["concept:name"].apply(activity_change)

# The following function change time format
def time_change(row):
  
  if len(row) == 32:
    dateTime = row[:19]
    zone = row[26:]
    newTime = dateTime+zone
    return newTime
  else:
    return row

df_clean["time:timestamp"] = df_clean["time:timestamp"].apply(time_change)
df_clean["time:timestamp"] = pd.to_datetime(df_clean["time:timestamp"],)
df_clean["case:concept:name"] = df_clean["case:concept:name"].astype(str)


In [0]:

# Save all trace id and activity belongs to a trace into dictionary
def activity(df_eve,unique_id):
  
  traces = {}
  
  for i in unique_id:
    list_a = []
    row_filter = df_eve.loc[df_eve['case:concept:name']==i,:]
    row_filter  = row_filter.reset_index(drop=True)
    
    for n in range(len(row_filter)):
      list_a.append(row_filter.loc[row_filter['case:concept:name']==i,'concept:name'][n])
    traces[i] = list_a
    
  return traces


# Filtering Distinct trace only
def dist_trace(df_eve,unique_id):
  
  Distinct_trace = []
  Distinct_trace_id = []
  
  # Calling activity method
  traces_list = activity(df_eve,unique_id)
  
  for k, v in traces_list.items():
      if v not in Distinct_trace:
        Distinct_trace.append(v)
        Distinct_trace_id.append(k)
        
  return Distinct_trace_id,Distinct_trace

unique_id = df_clean["case:concept:name"].unique()
dist_trace_id, dist_traces = dist_trace(df_clean,unique_id)

# Activity comination
def combination_of_activity(act_rename):
  # The following functions gets combination of events
  activity_name = act_rename

  Activity_Name = []
  for i in range(len(activity_name)):
    for c in range(len(activity_name)):
      Activity_Name.append([activity_name[i],activity_name[c]])
      #Activity_Name.append([activity_name[c],activity_name[i]])

  Activity_combination = []
  for combination in Activity_Name:
    if combination not in Activity_combination:
      Activity_combination.append(combination)
      
  return Activity_combination

# Count the combination
def combination_counter(trace_list,activity_combination):
  counts_of_combination = {}

  for trace_activity in trace_list:
  
    Activity_name = []
    for i in range(len(trace_activity)-1):
        Activity_name.append([trace_activity[i],trace_activity[i+1]])

    for c in Activity_name:
      if c in activity_combination:
        k =  repr(c)
        if k not in counts_of_combination.keys():
          counts_of_combination[k] = 1
        else:
          counts_of_combination[k] += 1
        
  return counts_of_combination

# Frequency of single activity
def activity_count(df_eve,unique_id,fre_act):
  
  Freq_act = 0
  for i in unique_id:
    row_filter = df_eve.loc[df_eve['case:concept:name']==i,:]
    row_filter  = row_filter.reset_index(drop=True)
    
    if fre_act:
      len_act = len(row_filter.loc[row_filter["concept:name"]==fre_act,"concept:name"])
      Freq_act +=len_act
    
  return Freq_act

freq_activity = {}
for i in act_rename:
  frequency_act = activity_count(df_clean,unique_id,i)
  freq_activity[i] = frequency_act
  #print("Frequency of {}:{}".format(i,frequency_act))
  
Activity_combi = combination_of_activity(act_rename)
CountsofComb = combination_counter(dist_traces,Activity_combi)

In [0]:
def prob_of_activity(CountsofComb,freq_activity,threshold_value):
  ab_a = []
  ab_b = []
  comman = []

  sorted_d = dict( sorted(CountsofComb.items(), key=operator.itemgetter(1),reverse=True))

  # finding the probability of activity given a
  for k , v in sorted_d.items():
    if k[2:5] in freq_activity:
      prob = round((v / freq_activity[k[2:5]]),3)  
      if prob >= threshold_value:
        ab_a.append(k)
        #print(k, ":", prob)

  # finding the probability of acitivity given b
  for k , v in sorted_d.items():
    if k[2:5] in freq_activity:
      prob1 = round((v / freq_activity[k[9:12]]),3)
      if prob1 >= threshold_value:
        ab_b.append(k)
        #print(k, ":", prob1)      

  for i in ab_a:
    if i in ab_b:
      comman.append(i)
      #print(i)
      
  return comman

In [0]:
def event_dropper(df_eve, activity_name, new_activity_name):
   
  unique_id = df_eve["case:concept:name"].unique()
  df_final = pd.DataFrame(columns=df_eve.columns)
  
  for i in unique_id: 
    event_index = []
    row_filter = df_eve.loc[df_eve['case:concept:name']==i,:]
    row_filter  = row_filter.reset_index(drop=True)
      
    
    for event in range(len(row_filter.loc[:,'concept:name'])-1):
      
      if activity_name[0] == activity_name[1]:
        if (activity_name[0] == row_filter.loc[event,'concept:name']) & (activity_name[1] == row_filter.loc[event + 1,'concept:name']):
          event_index.append(event)
        
      elif (activity_name[0] == row_filter.loc[event,'concept:name']) & (activity_name[1] == row_filter.loc[event + 1,'concept:name']):
        event_index.append(event)
        row_filter.loc[event + 1,'concept:name']=row_filter.loc[event + 1,'concept:name'].replace(activity_name[1],new_activity_name)
       

    row_filter.drop(row_filter.index[event_index],inplace=True)
    df_final = pd.concat([df_final, row_filter],ignore_index=True)
          
  return df_final

In [0]:
def remove_self_loop(df,file_Num):
  same_act = []
  act_rename = df["concept:name"].unique()
  unique_id = df["case:concept:name"].unique()
  dist_trace_id, dist_traces = dist_trace(df,unique_id)
  Activity_combi = combination_of_activity(act_rename)
  CountsofComb1 = combination_counter(dist_traces,Activity_combi)

  sorted_df = dict( sorted(CountsofComb1.items(), key=operator.itemgetter(1),reverse=True))
  for k , v in sorted_df.items():
    #print(k, ":", v)
    if k[2:5]==k[9:12]:
      same_act.append(k)
      #print(k, ":", v)
  
  new_a = same_act[0]
  df1 = event_dropper(df, new_a, None)
  for a in range(1,len(same_act)):
    new_ac = same_act[a]
    df1 = event_dropper(df1,new_ac,None)
  
  # Converting csv file to log file
  log = conversion_factory.apply(df1)
  # Saving log into xes file
  xes_file = '/content/drive/My Drive/Eventlog/Benchmark/seperate file/SEPSIS_{}.xes'.format(file_Num)
  xes_exporter.export_log(log, xes_file)


In [0]:
import numpy as np
threshold_values = np.arange(0.1, 1, 0.1)
n = 1
for t_v in threshold_values:
  comman_activity = prob_of_activity(CountsofComb,freq_activity,t_v)
  if len(comman_activity)>=1:
    
    # first combination
    if comman_activity[0][0] == comman_activity[0][1]:      
      df = event_dropper(df_clean, comman_activity[0],None)
    else:
      df = event_dropper(df_clean, comman_activity[0],"BBB")
      df["concept:name"] = df["concept:name"].apply(lambda x: re.sub(comman_activity[0][2:5], "BBB", x))
      df["concept:name"] = df["concept:name"].apply(lambda x: re.sub(comman_activity[0][2:5], "BBB", x))
      
    # Remaining combination
    for i in range(1,len(comman_activity)):
      if comman_activity[i][0] == comman_activity[i][1]:
        df = event_dropper(df, comman_activity[i],None)
      else: 
        df = event_dropper(df, comman_activity[i],"BB{}".format(i))
        df["concept:name"] = df["concept:name"].apply(lambda x: re.sub(comman_activity[i][2:5], "BB{}".format(i), x))
        df["concept:name"] = df["concept:name"].apply(lambda x: re.sub(comman_activity[i][2:5], "BB{}".format(i), x))
        
  remove_self_loop(df,n)
  n +=1

